| Esigenza tipica                                                  | Funzionalità Pandas                     | Quando serve                                                                |
| ---------------------------------------------------------------- | --------------------------------------- | --------------------------------------------------------------------------- |
| 1. **Importare** formati eterogenei (CSV, Excel, TSV, JSON, SQL) | `read_*()`                              | Ogni volta che i dati provengono da laboratori, EHR, registry o dispositivi |
| 2. **Pulire** dati sporchi o parziali                            | `dropna`, `fillna`, `astype`, `replace` | Prima di qualsiasi statistica o ML                                          |
| 3. **Ristrutturare** il dataset                                  | `melt`, `pivot`, `stack/unstack`        | Per allineare misure longitudinali o matrici di espressione                 |
| 4. **Filtrare & sottogruppi**                                    | Boolean indexing, `query`, `groupby`    | Focus su un fenotipo, un trattamento, un gene                               |
| 5. **Aggregare & statistiche rapide**                            | `groupby().agg()`, `describe`           | Report clinici, cruscotti QC                                                |
| 6. **Merge/Join** di più fonti                                   | `merge`, `concat`                       | Integrare imaging + clinica + omics                                         |
| 7. **Time series**                                               | `to_datetime`, `resample`, `rolling`    | Wearable, follow-up, ICU waveform                                           |


### 🏥 1 | Analisi di dataset clinici multifattoriali
(= studiare dati sanitari considerando più variabili contemporaneamente, come età, sesso, genetica, stile di vita, parametri fisiologici ...)

Scenario
Cartella clinica con: età, sesso, BMI, pressione S/D, HbA1c, diagnosi, outcome.

In [ ]:
import pandas as pd
df = pd.read_csv("registry_diabete.csv")
df["diagnosi"] = df["diagnosi"].str.upper().str.strip()  # normalizzo
stats = (df.groupby("diagnosi")[["HbA1c", "pressione_S", "pressione_D"]]
           .mean()
           .round(2))

Utilità:
- Pulizia nomenclatura diagnosi
- Statistiche descrittive per reparto
- Individuazione outlier (df[df["HbA1c"]>12])
- Esportazione rapida in Excel per il board clinico (stats.to_excel("summary.xlsx"))

{Tabella heat-map dei valori medi di HbA1c per diagnosi}

### 🧬 2 | Trascrittomica / RNA-seq
Scenario
Matrice genes × samples (>30 000 × 200).

In [ ]:
expr = pd.read_parquet("rna_seq.parquet")  # veloce su big data
# Filtra geni espressi
expr = expr.loc[expr.mean(axis=1) > 1]
corr = expr.T.corr(method="spearman")

Utilità:
- Lettura in Parquet → RAM-efficient
- Filtri basati su media/varianza
- Calcolo di reti di co-espressione
- Output diretto verso Seaborn/Scanpy

### 🧠 3 | Imaging e Quantificazione
Scenario
CSV export da 3D Slicer: ID paziente, ROI, volume (mm³), densità (HU).

In [ ]:
img = pd.read_csv("slicer_ROI_metrics.csv")
lesioni = img.query("volume_mm3 > 500 & ROI == 'tumor'")
trend = (lesioni.groupby("patient_id")
                 .apply(lambda g: g.sort_values("scan_date")
                                    .assign(delta=g["volume_mm3"].diff())))


Utilità:
- Filtraggio rapido su soglia volumetrica
- Calcolo delta-volume fra follow-up
- Facilita il merge con outcome clinici

### 🧫 4 | High-Content Screening (96-well, 384-well)
Scenario
Assorbanza OD a 0 h, 24 h, 48 h per 300 composti.

In [ ]:
raw = pd.read_excel("HCS_readout.xlsx")
pivot = raw.pivot_table(values="OD",
                        index=["compound_id"],
                        columns=["time_h"])
pivot["growth_%"] = (pivot[48] - pivot[0]) / pivot[0] * 100
top_hits = pivot.nlargest(10, "growth_%")

Utilità
- pivot_table rende il dataset “wide” per calcolo Δ
- Rank dei top-hits per successiva validazione
- Integrare annotazioni chimiche con merge

{Foto di piastra 384-well con pozzi colorimetrici + barplot crescita}

### 💉 5. Studi di Coorte Longitudinali
Scenario:
Dataset con pazienti seguiti per 5 anni, misurando outcome clinici (es. progressione di malattia, sopravvivenza, esiti).

In [ ]:
df["data_visita"] = pd.to_datetime(df["data_visita"])
df.sort_values(["paziente_id", "data_visita"], inplace=True)


cohort = pd.read_csv("coorte_cardiologia.csv", parse_dates=["visit_date"])
cohort.sort_values(["patient_id", "visit_date"], inplace=True)
cohort["years_from_baseline"] = (cohort["visit_date"] -
                                 cohort.groupby("patient_id")["visit_date"].transform("first")
                                ).dt.days / 365.25

Utilità:
- Ordinamento e calcolo di intervalli temporali
- Analisi per paziente o sottogruppo
- Calcolo della progressione (delta valori nel tempo)

{Grafico temporale con andamento di biomarcatori in un paziente nel tempo}